In [3]:
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
from openai import OpenAI

import os
from tqdm import tqdm


In [ ]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [5]:
df = pd.read_csv('COI_Grunenthal.csv')
df['authors_index'] = df['authors_index'].str.replace(r'\d+', '', regex=True)
df_sample = df
df_sample

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,doi,url,href,coi,grants,authors_index,abstract,affiliation,company
0,30719195,Adherence to tamoxifen in breast cancer patien...,"Schulz M, Klopp-Schulze L, Keilhack S, Meyer S...",Can Pharm J (Ott). 2018 Dec 20;152(1):28-34. d...,Schulz M,Can Pharm J (Ott),2018,2019/02/06,PMC6346334,NaN,10.1177/1715163518815720,https://pubmed.ncbi.nlm.nih.gov/30719195/,/30719195/,Conflict of interest:CK declares receiving res...,NaN,"Madlene Schulz , Lena Klopp-Schulze , Stefan...",NaN,Affiliation 1 Department of Clinical Pharmacy ...,Grünenthal
1,29513392,Cannabis-based medicines for chronic neuropath...,"Mücke M, Phillips T, Radbruch L, Petzke F, Häu...",Cochrane Database Syst Rev. 2018 Mar 7;3(3):CD...,Mücke M,Cochrane Database Syst Rev,2018,2018/03/08,PMC6494210,NaN,10.1002/14651858.CD012182.pub2,https://pubmed.ncbi.nlm.nih.gov/29513392/,/29513392/,MM: none known; MM is a specialist in palliati...,NaN,"Martin Mücke , Tudor Phillips, Lukas Radbruch...",Background: This review is one of a series on ...,Affiliation 1 Department of Palliative Medicin...,Grünenthal
2,29761479,Antidepressants for insomnia in adults,"Everitt H, Baldwin DS, Stuart B, Lipinska G, M...",Cochrane Database Syst Rev. 2018 May 14;5(5):C...,Everitt H,Cochrane Database Syst Rev,2018,2018/05/16,PMC6494576,NaN,10.1002/14651858.CD010753.pub2,https://pubmed.ncbi.nlm.nih.gov/29761479/,/29761479/,HE: has no interests to declare. DB: on behalf...,NaN,"Hazel Everitt , David S Baldwin, Beth Stuart,...",Background: Insomnia disorder is a subjective ...,Affiliation 1 Primary Care and Population Scie...,Grünenthal
3,29864216,Continuous intravenous perioperative lidocaine...,"Weibel S, Jelting Y, Pace NL, Helf A, Eberhart...",Cochrane Database Syst Rev. 2018 Jun 4;6(6):CD...,Weibel S,Cochrane Database Syst Rev,2018,2018/06/05,PMC6513586,NaN,10.1002/14651858.CD009642.pub3,https://pubmed.ncbi.nlm.nih.gov/29864216/,/29864216/,Stephanie Weibel: none known. Yvonne Jelting: ...,NaN,"Stephanie Weibel , Yvonne Jelting, Nathan L P...",Background: The management of postoperative pa...,Affiliation 1 Department of Anaesthesia and Cr...,Grünenthal
4,29293261,Treatment of fatigue in amyotrophic lateral sc...,"Gibbons C, Pagnini F, Friede T, Young CA.",Cochrane Database Syst Rev. 2018 Jan 2;1(1):CD...,Gibbons C,Cochrane Database Syst Rev,2018,2018/01/03,PMC6494184,NaN,10.1002/14651858.CD011005.pub2,https://pubmed.ncbi.nlm.nih.gov/29293261/,/29293261/,CG: no conflicts of interest. FP: no conflicts...,NaN,"Chris Gibbons , Francesco Pagnini, Tim Friede...",Background: Amyotrophic lateral sclerosis (ALS...,"Affiliation 1 The Primary Care Unit, Universit...",Grünenthal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,37942358,The Endocrine Treatment Landscape for Patients...,"Nabieva N, Altmann F, Apel K, Baerens DT, Beha...",Geburtshilfe Frauenheilkd. 2023 Jul 18;83(9):1...,Nabieva N,Geburtshilfe Frauenheilkd,2023,2023/11/09,PMC10629492,NaN,10.1055/a-2100-0643,https://pubmed.ncbi.nlm.nih.gov/37942358/,/37942358/,"Conflict of Interest N.N., C.Q., K.F. and J.K....",NaN,"Naiba Nabieva , Falk Altmann , Klaus Apel ,...",Introduction: While premenopausal patients wit...,"Affiliations 1 Novartis Pharma GmbH, Nürnberg,...",Grünenthal
1533,38186634,Brain ventricle and choroid plexus morphology ...,"Murck H, Fava M, Cusin C, Fatt CC, Trivedi M.",Brain Behav Immun Health. 2023 Dec 20;35:10071...,Murck H,Brain Behav Immun Health,2023,2024/01/08,PMC10767278,NaN,10.1016/j.bbih.2023.100717,https://pubmed.ncbi.nlm.nih.gov/38186634/,/38186634/,The authors declare the following financial in...,Grants and funding U01 MH092221/MH/NIMH NIH HH...,"Harald Murck , Maurizio Fava , Cristina Cusi...",Recent observations suggest a role of the volu...,Affiliations 1 Dept. of Psychiatry and Psychot...,Grünenthal
1534,37289165,[Pain in the vulnerable person. The psychologi...,Serra É.,Rev Prat. 2023 Apr;73

In [6]:
class COI(BaseModel):
    author: str = Field(description="The name or initials of the author reporting the conflict of interest.")
    organizations: List[str] = Field(description="List of organizations associated with the author.")
    sentence: str = Field(description="The entire sentence from the article mentioning the conflict of interest.") 

class COIList(BaseModel):
    cois: List[COI] = Field(description="List of COI entries.")

def extract_coi_info(coi_text: str, author_names: str) -> List[dict]:
    try:
        messages = [
            {"role": "system", "content": f"Extract the conflict of interests for each author in json format. The author names may be abbreviated. The author list is {author_names} - use their full names in the json."},
            {"role": "user", "content": coi_text},
        ]
        
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=messages,
            response_format=COIList,
            temperature=0.0,
        )
        
        event = completion.choices[0].message.parsed
        event_dict = event.dict()        
        return event_dict['cois']
    
    except Exception as e:
        print(f"Error processing COI text: {e}")
        return []



In [ ]:
chunk_size = 50  
output_folder = "output_folder_grunenthal/"  
start_chunk = 1
cache_file = "processed_chunks_cache_2.txt" 

os.makedirs(output_folder, exist_ok=True)

if os.path.exists(cache_file):
    with open(cache_file, "r") as f:
        processed_chunks = set(int(line.strip()) for line in f)
else:
    processed_chunks = set()

tqdm.pandas()

for i in range(start_chunk * chunk_size, len(df_sample), chunk_size):
    chunk_number = i // chunk_size + 1

    if chunk_number in processed_chunks:
        print(f"Skipping chunk {chunk_number} (already processed)")
        continue

    chunk = df_sample.iloc[i:i + chunk_size]   

    chunk['coi_info'] = chunk.progress_apply(
        lambda row: extract_coi_info(row['coi'], row['authors_index']),
        axis=1
    )

    chunk.to_csv(f"{output_folder}chunk_{chunk_number}.csv", index=False)
    print(f"Saved processed chunk {chunk_number}")

    with open(cache_file, "a") as f:
        f.write(f"{chunk_number}\n")


  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/8k/kp_pwhv96pjcp_nh49ycpw0c0000gn/T/ipykernel_22269/2661343855.py:24: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  event_dict = event.dict()
100%|██████████| 50/50 [26:40<00:00, 32.00s/it] 

Saved processed chunk 2
Skipping chunk 3 (already processed)
Skipping chunk 4 (already processed)
Skipping chunk 5 (already processed)
Skipping chunk 6 (already processed)
Skipping chunk 7 (already processed)
Skipping chunk 8 (already processed)
Skipping chunk 9 (already processed)
Skipping chunk 10 (already processed)
Skipping chunk 11 (already processed)
Skipping chunk 12 (already processed)
Skipping chunk 13 (already processed)
Skipping chunk 14 (already processed)
Skipping chunk 15 (already processed)
Skipping chunk 16 (already processed)
Skipping chunk 17 (already processed)
Skipping chunk 18 (already processed)
Skipping chunk 19 (already processed)
Skipping chunk 20 (already processed)
Skipping chunk 21 (already processed)
Skipping chunk 22 (already processed)
Skipping chunk 23 (already processed)
Skipping chunk 24 (already processed)
Skipping chunk 25 (already processed)
Skipping chunk 26 (already processed)
Skipping chunk 27 (already processed)
Skipping chunk 28 (already proces


/var/folders/8k/kp_pwhv96pjcp_nh49ycpw0c0000gn/T/ipykernel_22269/3922766337.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['coi_info'] = chunk.progress_apply(


In [ ]:
import os


In [14]:
processed_chunks = []
for file in os.listdir(output_folder):
    if file.endswith(".csv"):
        processed_chunks.append(pd.read_csv(os.path.join(output_folder, file)))

df_processed = pd.concat(processed_chunks, ignore_index=True)
df_processed

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,doi,url,href,coi,grants,authors_index,abstract,affiliation,company,coi_info
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,"[{'author': 'Jose Maria Remes-Troche', 'organi..."
1,34575456,Similar Piperacillin/Tazobactam Target Attainm...,"Busse D, Simon P, Petroff D, Dorn C, Schmitt L...",Pharmaceutics. 2021 Aug 31;13(9):1380. doi: 10...,Busse D,Pharmaceutics,2021,2021/09/28,PMC8464843,NaN,10.3390/pharmaceutics13091380,https://pubmed.ncbi.nlm.nih.gov/34575456/,/34575456/,C.K. and W.H. report grants from an industry c...,Grants and funding Open Access Funding/Freie U...,"David Busse , Philipp Simon , David Petrof...",Precision dosing of piperacillin/tazobactam in...,Affiliations 1 Department of Clinical Pharmacy...,Grünenthal,"[{'author': 'Charlotte Kloft', 'organizations'..."
2,34840785,To what extent are we confident that tapentado...,"Forget P, Vermeersch M.",Br J Pain. 2021 Nov;15(4):380-387. doi: 10.117...,Forget P,Br J Pain,2021,2021/11/29,PMC8611300,NaN,10.1177/2049463720945289,https://pubmed.ncbi.nlm.nih.gov/34840785/,/34840785/,Conflict of interest: The author(s) declared t...,NaN,"Patrice Forget , Mathieu Vermeersch",Background: A confidence evaluation helps to m...,Affiliations 1 Institute of Applied Health Sci...,Grünenthal,"[{'author': 'Patrice Forget', 'organizations':..."
3,33999006,Efficacy and Safety of Drug Combinations for C...,"Mohiuddin M, Park R, Wesselmann U, Pukall C, J...",JMIR Res Protoc. 2021 May 17;10(5):e21909. doi...,Mohiuddin M,JMIR Res Protoc,2021,2021/05/17,PMC8167620,NaN,10.2196/21909,https://pubmed.ncbi.nlm.nih.gov/33999006/,/33999006/,Conflicts of Interest: UW reports research gra...,Grants and funding R01 DK066641/DK/NIDDK NIH H...,"Mohammed Mohiuddin , Rex Park , Ursula Wesse...",Background: Chronic pelvic pain with various e...,Affiliations 1 Department of Anesthesiology & ...,Grünenthal,"[{'author': 'Ursula Wesselmann', 'organization..."
4,33151500,Bumetanide Oral Liquid Formulation for the Tre...,"Crutel V, Lambert E, Penelaud PF, Albarrán Sev...",J Autism Dev Disord. 2021 Aug;51(8):2959-2972....,Crutel V,J Autism Dev Disord,2021,2020/11/05,PMC8254707,NaN,10.1007/s10803-020-04709-8,https://pubmed.ncbi.nlm.nih.gov/33151500/,/33151500/,"VC, EL, P-FP, CAS, SK, SG, and MB are employee...",NaN,"Véronique Crutel , Estelle Lambert , Pierre-...",There are currently no approved pharmacologica...,Affiliations 1 Neuro Immuno-Inflammation Thera...,Grünenthal,"[{'author': 'Véronique Crutel', 'organizations..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,32500375,"Chance or challenge, spoilt for choice? New re...","Dohrn MF, Auer-Grumbach M, Baron R, Birklein F...",J Neurol. 2021 Oct;268(10):3610-3625. doi: 10....,Dohrn MF,J Neurol,2021,2020/06/06,PMC8463516,NaN,10.1007/s00415-020-09962-6,https://pubmed.ncbi.nlm.nih.gov/32500375/,/32500375/,"MFD, FB, FEL, CG, NG, EH, MS, JSch, WSM, CS, M...",NaN,"Maike F Dohrn , Michaela Auer-Grumbach , Ral...",Hereditary transthyretin amyloidosis is caused...,Affiliations 1 Neuromuscular Outpatient Clinic...,Grünenthal,"[{'author': 'Maike F Dohrn', 'organizations': ..."
1533,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,10.1136/annrheumdis-2020-219498,https://pubmed.ncbi.nlm.nih.gov/335044

In [15]:
df_processed['coi_info'] = df_processed['coi_info'].apply(eval)
df_exploded = df_processed.explode('coi_info')
df_exploded

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,doi,url,href,coi,grants,authors_index,abstract,affiliation,company,coi_info
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,"{'author': 'Jose Maria Remes-Troche', 'organiz..."
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,"{'author': 'Enrique Coss-Adame', 'organization..."
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,"{'author': 'Mercedes Amieva-Balmori', 'organiz..."
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,{'author': 'José Antonio Velarde-Ruiz Velasco'...
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,10.1136/bmjgast-2021-000729,https://pubmed.ncbi.nlm.nih.gov/34531245/,/34531245/,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,"{'author': 'Paulo César Gómez-Castaños', 'orga..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,10.1111/nmo.14136,https://pubmed.ncbi.nlm.nih.gov/33934444/,/33934444/,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,"{'author': 'Joanna W Kruimel', 'organizations'..."
1536,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,10.1111/nmo.14136,https://pubm

In [16]:
df_exploded = pd.concat([df_exploded.drop(['coi_info'], axis=1), df_exploded['coi_info'].apply(pd.Series)], axis=1)
df_exploded

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,coi,grants,authors_index,abstract,affiliation,company,0,author,organizations,sentence
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,"[Takeda, Asofarma, Biocodex, Medtronic, Carnot...",JMR-T is a member of the advisory board of Tak...
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Enrique Coss-Adame,"[Takeda, Asofarma, Medtronic, Carnot, Grünenthal]","EC-A has been a speaker for Takeda, Asofarma, ..."
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Mercedes Amieva-Balmori,"[Takeda, Asofarma]",MA-B has been a speaker for Takeda and Asofarma.
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,José Antonio Velarde-Ruiz Velasco,[],"JAV-RV, PCG-C, RF-R, OG-E, MCR-L, CD-R, SMMP-G..."
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Paulo César Gómez-Castaños,[],"JAV-RV, PCG-C, RF-R, OG-E, MCR-L, CD-R, SMMP-G..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Joanna W Kruimel,[],J.W.K.: none to declare.
1536,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Ga

In [23]:
df_exploded.to_csv('COI_Grunenthal_extracted.csv', index=False)

In [25]:
df_exploded = pd.read_csv('COI_Grunenthal_extracted.csv')
df_exploded

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,coi,grants,authors_index,abstract,affiliation,company,0,author,organizations,sentence
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,takeda,JMR-T is a member of the advisory board of Tak...
1,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,asofarma,JMR-T is a member of the advisory board of Tak...
2,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,biocodex,JMR-T is a member of the advisory board of Tak...
3,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,medtronic,JMR-T is a member of the advisory board of Tak...
4,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,carnot,JMR-T is a member of the advisory board of Tak...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88230,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,grunenthal,"D.K.: grants from Will Pharma, Allergan, Grune..."
88231,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,zonmw,"D.K

In [ ]:
df_exploded[df_exploded['pmid'] == 34531245]

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,coi,grants,authors_index,abstract,affiliation,company,0,author,organizations,sentence
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,takeda,JMR-T is a member of the advisory board of Tak...
1,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,asofarma,JMR-T is a member of the advisory board of Tak...
2,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,biocodex,JMR-T is a member of the advisory board of Tak...
3,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,medtronic,JMR-T is a member of the advisory board of Tak...
4,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,carnot,JMR-T is a member of the advisory board of Tak...
5,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,alfasigma,JMR-T is a member of the advisory board of Tak...
6,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Enrique Coss-Adame,takeda,"EC-A has been a speaker for Takeda, Asofarma, ..."
7,34531245,Incidence of 'new-onset' constip

# For each pmid/author i am going to scrape the author id

In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import os
import time

def get_author_ids_and_labels(pmid, retries=3):
    url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}"
    for attempt in range(retries):
        try:
            response = requests.get(url)
            if response.status_code != 200:
                print(f"Failed to retrieve data for PMID: {pmid}, Status code: {response.status_code}")
                return []

            soup = BeautifulSoup(response.content, 'html.parser')
            authors = []

            for author in soup.find_all('span', class_='authors-list-item'):
                author_link = author.find('a', href=True)
                if author_link and 'cauthor_id' in author_link['href']:
                    author_name = author_link['href'].split('term=')[-1].split('&')[0].replace('+', ' ')
                    data_ga_label = author_link.get('data-ga-label', '')
                    authors.append({'name': author_name, 'data_ga_label': data_ga_label})

            return authors
        except requests.exceptions.RequestException as e:
            print(f"Error retrieving data for PMID: {pmid}, Attempt: {attempt + 1}, Error: {e}")
            time.sleep(2)  
    return []

pmids = df_exploded['pmid'].unique()
all_authors = []

output_folder = 'author_chunks'
os.makedirs(output_folder, exist_ok=True)

chunk_size = 100  
chunk_number = 13 

start_index = chunk_size * (chunk_number - 1)

for i in tqdm(range(start_index, len(pmids), chunk_size), desc="Processing chunks"):
    chunk_pmids = pmids[i:i + chunk_size]
    chunk_authors = []
    
    for pmid in tqdm(chunk_pmids, desc=f"Processing chunk {chunk_number}", leave=False):
        authors = get_author_ids_and_labels(pmid)
        for author in authors:
            author['pmid'] = pmid  
        chunk_authors.extend(authors)
    
    df_chunk = pd.DataFrame(chunk_authors)
    chunk_file_path = os.path.join(output_folder, f'authors_chunk_{chunk_number}.csv')
    df_chunk.to_csv(chunk_file_path, index=False)
    chunk_number += 1

print(f"Processed {chunk_number - 1} chunks and saved in '{output_folder}' folder.")

Processing chunks: 100%|██████████| 4/4 [03:32<00:00, 53.20s/it]

Processed 16 chunks and saved in 'author_chunks' folder.


In [147]:
df_authors.name.value_counts()

Danese S             226
Peyrin-Biroulet L    138
D%27Amico F           94
Reinisch W            84
Kloft C               78
                    ... 
Custovic A             1
Courbis AL             1
de Sousa JC            1
Chkhartishvili E       1
Gault N                1
Name: name, Length: 11621, dtype: int64

In [154]:
# read all the chunks and concatenate them into a single dataframe
author_chunks = []
for file in os.listdir(output_folder):
    if file.endswith(".csv"):
        author_chunks.append(pd.read_csv(os.path.join(output_folder, file)))

df_authors = pd.concat(author_chunks, ignore_index=True)
df_authors



,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,abstract,affiliation,company,0,author,organizations,sentence,name,data_ga_label,coi_reason_info
0,32500375,"Chance or challenge, spoilt for choice? New re...","Dohrn MF, Auer-Grumbach M, Baron R, Birklein F...",J Neurol. 2021 Oct;268(10):3610-3625. doi: 10....,Dohrn MF,J Neurol,2021,2020/06/06,PMC8463516,NaN,...,Hereditary transthyretin amyloidosis is caused...,Affiliations 1 Neuromuscular Outpatient Clinic...,Grünenthal,NaN,Ralf Baron,grünenthal gmbh,RB reports grants/research support from Pfizer...,Baron R,Ralf Baron,"['Grants/Research support', 'Speaker fees', 'C..."
1,32500375,"Chance or challenge, spoilt for choice? New re...","Dohrn MF, Auer-Grumbach M, Baron R, Birklein F...",J Neurol. 2021 Oct;268(10):3610-3625. doi: 10....,Dohrn MF,J Neurol,2021,2020/06/06,PMC8463516,NaN,...,Hereditary transthyretin amyloidosis is caused...,Affiliations 1 Neuromuscular Outpatient Clinic...,Grünenthal,NaN,Jens Schmidt,grünenthal gmbh,JS reports personal fees from Grünenthal GmbH ...,Schmidt J,Jens Schmidt,"['Consulting fees', 'Travel support']"
2,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Elsa F Mateus,grünenthal s.a.,EFM reports that LPCDR received support for sp...,Mateus EF,Elsa F Mateus,"['Grants', 'Non-financial support']"
3,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Elsa F Mateus,grünenthal gmbh,EFM reports that LPCDR received support for sp...,Mateus EF,Elsa F Mateus,"['Grants', 'Non-financial support']"
4,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Thierry Thomas,grunenthal,TT reports personal fees for lectures and expe...,Thomas T,Thierry Thomas,"['Consulting fees', 'Speaking engagements', 'R..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,34593832,Preclinical development of a bispecific TNFα/I...,"Roberts KJ, Cubitt MF, Carlton TM, Rodrigues-D...",Sci Rep. 2021 Sep 30;11(1):19422. doi: 10.1038...,Roberts KJ,Sci Rep,2021,2021/10/01,PMC8484351,NaN,...,Anti-TNFα and anti-IL-23 antibodies are highly...,"Affiliations 1 VHsquared Ltd., 1 Lower Court, ...",Grünenthal,NaN,Thomas T MacDonald,grunenthal,T.T.M. received honoraria and/or research fund...,MacDonald TT,Thomas T MacDonald,"['Research grants', 'Honoraria']"
2096,34742252,Real-world evidence data on the monoclonal ant...,"Straube A, Stude P, Gaul C, Schuh K, Koch M.",J Headache Pain. 2021 Nov 6;22(1):133. doi: 10...,Straube A,J Headache Pain,2021,2021/11/07,PMC8572451,NaN,...,"Background: Erenumab, the first-in-class fully...","Affiliations 1 Department of Neurology, Univer...",Grünenthal,NaN,Charly Gaul,grünenthal,CG received honoraria for consulting and lectu...,Gaul C,Charly Gaul,"['Consulting fees', 'Speaking engagements']"
2097,34895130,Age- and frequency-dependent changes in dynami...,"Eren OE, Straube A, Schöberl F, Ruscheweyh R, ...",J Headache Pain. 2021 Dec 11;22(1):148. doi: 1...,Eren OE,J Headache Pain,2021,2021/12/13,PMC8903496,NaN,...,Objective: Patients with visual snow syndrome ...,"Affiliations 1 Department of Neurology, LMU Mu...",Grünenthal,NaN,C

In [93]:
df_exploded

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,coi,grants,authors_index,abstract,affiliation,company,0,author,organizations,sentence
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,takeda,JMR-T is a member of the advisory board of Tak...
1,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,asofarma,JMR-T is a member of the advisory board of Tak...
2,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,biocodex,JMR-T is a member of the advisory board of Tak...
3,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,medtronic,JMR-T is a member of the advisory board of Tak...
4,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,carnot,JMR-T is a member of the advisory board of Tak...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88230,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,grunenthal,"D.K.: grants from Will Pharma, Allergan, Grune..."
88231,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,zonmw,"D.K

In [84]:
# remove all the symbols from the author names in df_exploded
df_exploded['author'] = df_exploded['author'].str.replace('.', '')
df_exploded

/var/folders/8k/kp_pwhv96pjcp_nh49ycpw0c0000gn/T/ipykernel_22269/3402262465.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_exploded['author'] = df_exploded['author'].str.replace('.', '')


,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,coi,grants,authors_index,abstract,affiliation,company,0,author,organizations,sentence
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,takeda,JMR-T is a member of the advisory board of Tak...
1,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,asofarma,JMR-T is a member of the advisory board of Tak...
2,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,biocodex,JMR-T is a member of the advisory board of Tak...
3,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,medtronic,JMR-T is a member of the advisory board of Tak...
4,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,Competing interests: JMR-T is a member of the ...,NaN,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,carnot,JMR-T is a member of the advisory board of Tak...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88230,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,grunenthal,"D.K.: grants from Will Pharma, Allergan, Grune..."
88231,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"T.K.: personal fees from Will Pharma, outside ...",NaN,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,zonmw,"D.K

In [119]:
df_authors[df_authors['pmid'] == 36427185]

,name,data_ga_label,pmid
11368,Preiss Contreras Y,Yudith Preiss Contreras,36427185
11369,Ramos Salas X,Ximena Ramos Salas,36427185
11370,%C3%81vila Oliver C,Camila Ávila Oliver,36427185
11371,Saquimux Contreras MA,Miguel Alejandro Saquimux Contreras,36427185
11372,Mu%C3%B1oz Claro R,Rodrigo Muñoz Claro,36427185
11373,Canales Ferrada C,Claudio Canales Ferrada,36427185
11374,Preiss Contreras Y,Yudith Preiss Contreras,36427185
11375,Ramos Salas X,Ximena Ramos Salas,36427185
11376,%C3%81vila Oliver C,Camila Ávila Oliver,36427185
11377,Saquimux Contreras MA,Miguel Alejandro Saquimux Contreras,36427185


In [121]:
df_exploded[df_exploded['pmid'] == 36427185]['authors_index']

7478    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7479    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7480    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7481    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7482    Yudith Preiss Contreras  , Ximena Ramos Salas ...
                              ...                        
7540    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7541    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7542    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7543    Yudith Preiss Contreras  , Ximena Ramos Salas ...
7544    Yudith Preiss Contreras  , Ximena Ramos Salas ...
Name: authors_index, Length: 67, dtype: object

In [125]:
df_authors_unique = df_authors.drop_duplicates(subset=['data_ga_label', 'pmid', 'name'])

df_merged = pd.merge(df_exploded, df_authors_unique, left_on=['author', 'pmid'], right_on=['data_ga_label', 'pmid'], how='left')

print(f"df_exploded rows: {len(df_exploded)}")
print(f"df_merged rows: {len(df_merged)}")
df_merged

df_exploded rows: 88235
df_merged rows: 88235


,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,authors_index,abstract,affiliation,company,0,author,organizations,sentence,name,data_ga_label
0,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,takeda,JMR-T is a member of the advisory board of Tak...,Remes-Troche JM,Jose Maria Remes-Troche
1,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,asofarma,JMR-T is a member of the advisory board of Tak...,Remes-Troche JM,Jose Maria Remes-Troche
2,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,biocodex,JMR-T is a member of the advisory board of Tak...,Remes-Troche JM,Jose Maria Remes-Troche
3,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,medtronic,JMR-T is a member of the advisory board of Tak...,Remes-Troche JM,Jose Maria Remes-Troche
4,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Jose Maria Remes-Troche,carnot,JMR-T is a member of the advisory board of Tak...,Remes-Troche JM,Jose Maria Remes-Troche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88230,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,grunenthal,"D.K.: grants from Will Pharma, Allergan, Grune...",Keszthelyi D,Daniel Keszthelyi
88231,33934444,Psychometric evaluation of an experience sampl...,"Klaassen T, Smeets FGM, Vork L, Tack J, Talley...",Neurogastroenterol Motil. 2021 Sep;33(9):e1413...,Klaassen T,Neurogastroenterol Motil,2021,2021/05/02,PMC8519073,NaN,...,"Tim Klaassen , Fabienne G M Smeets , Lisa Vo...","Background: Due to important biases, conventio...",Affiliations 1 Division of Gastroenterology-He...,Grünenthal,NaN,Daniel Keszthelyi,zonmw,"D.K.: grants from Will Pharma, Allergan, Grune...",Keszthelyi D,Daniel Keszthelyi
88232,33934444,Psychometric evaluation of an experience sampl...

In [145]:
# df_merged where name is null
df_merged[df_merged['name'].isnull()].author.unique()

array(['Verónica Álvarez', 'Paulina Andrade Risso', 'Camila Avila',
       'Claudio Canales', 'Carolina Carrasco', 'Fernando Carrasco',
       'Ariel Cisternas Villalobos', 'Ada Cuevas', 'Samuel Durán Agüero',
       'Elizabeth Erdös Gattoni', 'Alex Escalona', 'Magdalena Farías',
       'Daniela Eugenia Gómez Pérez', 'María Isabel González Galvez',
       'Carolina González Reyes', 'Patricio Lamoza Kohan',
       'María José Leiva Velasco', 'Marcia López Aceitón',
       'Denisse Montt Saavedra', 'Manuel Moreno', 'Karin Papapietro',
       'Israel Podestá Donoso', 'Camila Rodriguez', 'Matías Sepúlveda',
       'Arturo Zumaeta Valenzuela', 'Paolo Maria Matricardi',
       'Zeljko Reiner', 'Robert W Gereau', 'Liam G Heaney', nan,
       'Tiago Teodoro', 'Akihiro Koreki', 'Anne Marthe Meppelink',
       'Simon Little', 'Glenn Nielsen', 'Antonella Macerollo',
       'Mark J Edwards', 'Joaquim J Ferreira', 'Isabel Pareés',
       'Anthony Lang', 'Katharina Gottwald', 'O Bouché', 'C Le Pécho

In [ ]:
df_merged.to_csv('COI_Grunenthal_merged.csv', index=False)

# Getting the relationship between the author and the COI company

In [ ]:
df_exploded = df_exploded.explode('organizations')
df_exploded['organizations'] = df_exploded['organizations'].str.lower()



In [127]:
grunenthal_df = df_merged[(df_merged['organizations'].str.contains('grünenthal|grunenthal', na=False)) ]
grunenthal_df

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,authors_index,abstract,affiliation,company,0,author,organizations,sentence,name,data_ga_label
10,34531245,Incidence of 'new-onset' constipation and asso...,"Remes-Troche JM, Coss-Adame E, Amieva-Balmori ...",BMJ Open Gastroenterol. 2021 Sep;8(1):e000729....,Remes-Troche JM,BMJ Open Gastroenterol,2021,2021/09/17,PMC8449841,NaN,...,"Jose Maria Remes-Troche , Enrique Coss-Adame ...",Objective: A minimum of physical activity and ...,Affiliations 1 Instituto de Investigaciones Mé...,Grünenthal,NaN,Enrique Coss-Adame,grünenthal,"EC-A has been a speaker for Takeda, Asofarma, ...",Coss-Adame E,Enrique Coss-Adame
25,34575456,Similar Piperacillin/Tazobactam Target Attainm...,"Busse D, Simon P, Petroff D, Dorn C, Schmitt L...",Pharmaceutics. 2021 Aug 31;13(9):1380. doi: 10...,Busse D,Pharmaceutics,2021,2021/09/28,PMC8464843,NaN,...,"David Busse , Philipp Simon , David Petrof...",Precision dosing of piperacillin/tazobactam in...,Affiliations 1 Department of Clinical Pharmacy...,Grünenthal,NaN,Charlotte Kloft,grünenthal,C.K. and W.H. report grants from an industry c...,Kloft C,Charlotte Kloft
36,34575456,Similar Piperacillin/Tazobactam Target Attainm...,"Busse D, Simon P, Petroff D, Dorn C, Schmitt L...",Pharmaceutics. 2021 Aug 31;13(9):1380. doi: 10...,Busse D,Pharmaceutics,2021,2021/09/28,PMC8464843,NaN,...,"David Busse , Philipp Simon , David Petrof...",Precision dosing of piperacillin/tazobactam in...,Affiliations 1 Department of Clinical Pharmacy...,Grünenthal,NaN,Wilhelm Huisinga,grünenthal,C.K. and W.H. report grants from an industry c...,Huisinga W,Wilhelm Huisinga
46,34840785,To what extent are we confident that tapentado...,"Forget P, Vermeersch M.",Br J Pain. 2021 Nov;15(4):380-387. doi: 10.117...,Forget P,Br J Pain,2021,2021/11/29,PMC8611300,NaN,...,"Patrice Forget , Mathieu Vermeersch",Background: A confidence evaluation helps to m...,Affiliations 1 Institute of Applied Health Sci...,Grünenthal,NaN,Patrice Forget,grünenthal,P.F. has been advisory board member for Grünen...,Forget P,Patrice Forget
50,33999006,Efficacy and Safety of Drug Combinations for C...,"Mohiuddin M, Park R, Wesselmann U, Pukall C, J...",JMIR Res Protoc. 2021 May 17;10(5):e21909. doi...,Mohiuddin M,JMIR Res Protoc,2021,2021/05/17,PMC8167620,NaN,...,"Mohammed Mohiuddin , Rex Park , Ursula Wesse...",Background: Chronic pelvic pain with various e...,Affiliations 1 Department of Anesthesiology & ...,Grünenthal,NaN,Ursula Wesselmann,grünenthal gmbh,UW reports research grants from NIH and serves...,Wesselmann U,Ursula Wesselmann
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88077,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,"Anja Strangfeld # , Martin Schäfer # , Milen...",Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Loreto Carmona,grünenthal gmbh,LC has not received fees or personal grants fr...,Carmona L,Loreto Carmona
88159,34017304,Magnetic Suppression of Perceptual Accuracy Is...,"Eren OE, Ruscheweyh R, Rauschel V, Eggert T, S...",Front Neurol. 2021 May 4;12:658857. doi: 10.33...,Eren OE,Front Neurol,2021,2021/05/21,PMC8129492,NaN,...,"Ozan E Eren , Ruth Ruscheweyh , Veronika Rau...",Objective: Patients with visual snow syndrome ...,"Affiliations 1 Department of Neurology, Ludwig...",Grünenthal,NaN,Christoph J Schankin,grünenthal,CS reports grants from Deutsche Migräne- und K...,Schankin CJ,Christoph J Schankin
88175,34063980,Novel Pharmacokinetic/Pharmacodynamic Paramete...,"Seeger J, Guenther S, Schaufler K, Heiden SE, ...",Antibiotics (Basel). 2021 May 21;10(6):615. do...,Seeger J,Antibiotics (Basel),2021,2021/06/02,PMC8224043,NaN,...,"Johanna Seeger , Sebastian Guenther , 

In [131]:
class COIReason(BaseModel):
    coi_reason: List[str] = Field(description="Reason(s) for the conflict of interest.")

def extract_coireason_info(author: str, organization: str, coi_text: str) -> List[dict]:
    try:
        messages = [
            {"role": "system", "content": f"Extract the nature of relationship between author: {author} and organization: {organization}."},
            {"role": "user", "content": coi_text},
        ]
        
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=messages,
            response_format=COIReason,
            temperature=0.0,
        )
        
        event = completion.choices[0].message.parsed
        event_dict = event.dict()        
        return event_dict['cois']
    
    except Exception as e:
        print(f"Error processing COI text: {e}")
        return []



In [132]:
from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List
import pandas as pd

tqdm.pandas()

class COIReason(BaseModel):
    coi_reason: List[str] = Field(description="Reason(s) for the conflict of interest.")

def extract_coireason_info(author: str, organization: str, coi_text: str) -> List[str]:
    try:
        messages = [
            {
                "role": "system",
                "content": (
                    f"Categorize the reasons for the conflict of interest (COI) between the author '{author}' "
                    f"and the organization '{organization}'. "
                    "Provide a concise list of categories such as 'Research grants', 'Consulting fees', "
                    "'Speaking engagements', 'Stock ownership', etc. Do not include additional text or explanations."
                )
            },
            {"role": "user", "content": coi_text},
        ]
        
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=messages,
            response_format=COIReason,
            temperature=0.0,
        )
        
        event = completion.choices[0].message.parsed
        event_dict = event.dict()        
        return event_dict.get('coi_reason', [])
    
    except Exception as e:
        print(f"Error processing COI text for author '{author}' and organization '{organization}': {e}")
        return []





In [136]:
import os
import pandas as pd
from tqdm import tqdm


output_folder = 'grunenthal_chunks'
os.makedirs(output_folder, exist_ok=True)

chunk_size = 10  
num_chunks = len(grunenthal_df) // chunk_size + 1

start_chunk = 30

for chunk_number in tqdm(range(start_chunk, num_chunks), desc="Processing chunks"):
    start_index = chunk_number * chunk_size
    end_index = start_index + chunk_size
    chunk_df = grunenthal_df.iloc[start_index:end_index].copy()
    
    chunk_df['coi_reason_info'] = chunk_df.progress_apply(
        lambda row: extract_coireason_info(row['author'], row['organizations'], row['coi'])
        if all(k in row for k in ['author', 'organizations', 'coi']) else [],
        axis=1
    )
    
    chunk_file_path = os.path.join(output_folder, f'grunenthal_chunk_{chunk_number}.csv')
    chunk_df.to_csv(chunk_file_path, index=False)

print(f"Processed chunks from {start_chunk} to {num_chunks - 1} and saved in '{output_folder}' folder.")

Processing chunks:   0%|          | 0/181 [00:00<?, ?it/s]/var/folders/8k/kp_pwhv96pjcp_nh49ycpw0c0000gn/T/ipykernel_22269/4112889907.py:34: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  event_dict = event.dict()
100%|██████████| 10/10 [00:08<00:00,  1.12it/s]:38<00:28, 14.12s/it]
1it [00:00, 289.86it/s]|█████████▉| 180/181 [43:47<00:12, 12.56s/it]
Processing chunks: 100%|██████████| 181/181 [43:47<00:00, 14.52s/it]

Processed chunks from 30 to 210 and saved in 'grunenthal_chunks' folder.


In [137]:
# read all the chunks and concatenate them into a single dataframe

coireason_chunks = []
for file in os.listdir(output_folder):
    if file.endswith(".csv"):
        coireason_chunks.append(pd.read_csv(os.path.join(output_folder, file)))

df_coireason = pd.concat(coireason_chunks, ignore_index=True)
df_coireason

,pmid,title,authors,citation,first_author,journal,publication_year,create_date,pmcid,nihms_id,...,abstract,affiliation,company,0,author,organizations,sentence,name,data_ga_label,coi_reason_info
0,32500375,"Chance or challenge, spoilt for choice? New re...","Dohrn MF, Auer-Grumbach M, Baron R, Birklein F...",J Neurol. 2021 Oct;268(10):3610-3625. doi: 10....,Dohrn MF,J Neurol,2021,2020/06/06,PMC8463516,NaN,...,Hereditary transthyretin amyloidosis is caused...,Affiliations 1 Neuromuscular Outpatient Clinic...,Grünenthal,NaN,Ralf Baron,grünenthal gmbh,RB reports grants/research support from Pfizer...,Baron R,Ralf Baron,"['Grants/Research support', 'Speaker fees', 'C..."
1,32500375,"Chance or challenge, spoilt for choice? New re...","Dohrn MF, Auer-Grumbach M, Baron R, Birklein F...",J Neurol. 2021 Oct;268(10):3610-3625. doi: 10....,Dohrn MF,J Neurol,2021,2020/06/06,PMC8463516,NaN,...,Hereditary transthyretin amyloidosis is caused...,Affiliations 1 Neuromuscular Outpatient Clinic...,Grünenthal,NaN,Jens Schmidt,grünenthal gmbh,JS reports personal fees from Grünenthal GmbH ...,Schmidt J,Jens Schmidt,"['Consulting fees', 'Travel support']"
2,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Elsa F Mateus,grünenthal s.a.,EFM reports that LPCDR received support for sp...,Mateus EF,Elsa F Mateus,"['Grants', 'Non-financial support']"
3,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Elsa F Mateus,grünenthal gmbh,EFM reports that LPCDR received support for sp...,Mateus EF,Elsa F Mateus,"['Grants', 'Non-financial support']"
4,33504483,Factors associated with COVID-19-related death...,"Strangfeld A, Schäfer M, Gianfrancesco MA, Law...",Ann Rheum Dis. 2021 Jul;80(7):930-942. doi: 10...,Strangfeld A,Ann Rheum Dis,2021,2021/01/28,PMC7843211,NaN,...,Objectives: To determine factors associated wi...,Collaborators COVID-19 Global Rheumatology All...,Grünenthal,NaN,Thierry Thomas,grunenthal,TT reports personal fees for lectures and expe...,Thomas T,Thierry Thomas,"['Consulting fees', 'Speaking engagements', 'R..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,34593832,Preclinical development of a bispecific TNFα/I...,"Roberts KJ, Cubitt MF, Carlton TM, Rodrigues-D...",Sci Rep. 2021 Sep 30;11(1):19422. doi: 10.1038...,Roberts KJ,Sci Rep,2021,2021/10/01,PMC8484351,NaN,...,Anti-TNFα and anti-IL-23 antibodies are highly...,"Affiliations 1 VHsquared Ltd., 1 Lower Court, ...",Grünenthal,NaN,Thomas T MacDonald,grunenthal,T.T.M. received honoraria and/or research fund...,MacDonald TT,Thomas T MacDonald,"['Research grants', 'Honoraria']"
2096,34742252,Real-world evidence data on the monoclonal ant...,"Straube A, Stude P, Gaul C, Schuh K, Koch M.",J Headache Pain. 2021 Nov 6;22(1):133. doi: 10...,Straube A,J Headache Pain,2021,2021/11/07,PMC8572451,NaN,...,"Background: Erenumab, the first-in-class fully...","Affiliations 1 Department of Neurology, Univer...",Grünenthal,NaN,Charly Gaul,grünenthal,CG received honoraria for consulting and lectu...,Gaul C,Charly Gaul,"['Consulting fees', 'Speaking engagements']"
2097,34895130,Age- and frequency-dependent changes in dynami...,"Eren OE, Straube A, Schöberl F, Ruscheweyh R, ...",J Headache Pain. 2021 Dec 11;22(1):148. doi: 1...,Eren OE,J Headache Pain,2021,2021/12/13,PMC8903496,NaN,...,Objective: Patients with visual snow syndrome ...,"Affiliations 1 Department of Neurology, LMU Mu...",Grünenthal,NaN,C

In [161]:
df_coireason['coi_reason_info'] 

0       ['Grants/Research support', 'Speaker fees', 'C...
1                   ['Consulting fees', 'Travel support']
2                     ['Grants', 'Non-financial support']
3                     ['Grants', 'Non-financial support']
4       ['Consulting fees', 'Speaking engagements', 'R...
                              ...                        
2095                     ['Research grants', 'Honoraria']
2096          ['Consulting fees', 'Speaking engagements']
2097    ['Research grants', 'Consulting fees', 'Speaki...
2098          ['Honorarium fees', 'Speaking engagements']
2099                                  ['Consulting fees']
Name: coi_reason_info, Length: 2100, dtype: object

In [138]:
df_coireason.to_csv('COI_Grunenthal_processed.csv', index=False)